## Boilerplate

In [1]:
%run ../config/initialize.ipynb

import json
import shutil

In [2]:
%load_ext autoreload
%autoreload 2

## TODO
* how to generalize to regression problems?
  * easier to create a separate process?
  
### Modeling ideas
* different labels (regression?)
* hyperparam tuning
* model selection
* feature/team combinations, e.g. H-A DVOA
* narrow down to certain weeks

## Model Prep

In [3]:
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 00_plan_and_ideas.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 01_time_date.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 02_teams.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 03_matchup.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 04_travel.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 05_homeaway.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 06_weather.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 07_line.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run 08_rankings.ipynb
# os.chdir('../modeling-football-outcomes/05_feature_engineering/')
# %run combine_features.ipynb

In [4]:
# ## SPREAD
# ranks = spark.table('dvoa').select(
#         'team_id','season','week_id','dave_or_wtddvoa',
#         'offensedvoa','defensedvoa','s_t_dvoa'
#     ).cache()

# game_feats = spark.table('labels.team_game_line_labels').select(
#         'game_id','season','week_id','team_id','is_home'
#     ).cache()

# features = game_feats.join(
#         ranks, on=['team_id','season','week_id']
#     ).fillna(-99999)
# labels = spark.table('labels.team_game_line_labels').select(
#     ## index
#     'game_id','team_id',
#     ## strata
#     'is_home','is_fav_sbr',
#     ## labels
#     'did_win','final_margin','did_cover_pfr',
#     'did_cover_sbr','did_cover_sbr_open'
# )

# assert features.count() == labels.count()

# features.write.mode('overwrite').saveAsTable('features.190320_test')
# labels.write.mode('overwrite').saveAsTable('labels.190320_test')

# ## OVER/UNDER
# features = spark.table('labels.over_under_labels').select(
#         'game_id','season','week_id', 'sbr_ou'
#     ).fillna(-99999).cache()

# labels = spark.table('labels.over_under_labels').select(
#     ## index
#     'game_id',
#     ## strata
#     ## label
#     'is_sbr_ou_over'
# )

# assert features.count() == labels.count()

# features.write.mode('overwrite').saveAsTable('features.190320_ou_test')
# labels.write.mode('overwrite').saveAsTable('labels.190320_ou_test')

## Set root dir path

In [5]:
ROOT='/Users/joshplotkin/Dropbox/data_science/'\
    'modeling-football-outcomes/models'
os.chdir(ROOT)

## Initialize
* set model ID
* remove this model ID's directory if it exists
* create directory

In [6]:
MODEL_ID = '0320_with_rankings_winner_20feats_noml'

if os.path.exists(MODEL_ID):
    print 'Directory models/{} **EXISTS**'.format(MODEL_ID)    
else:
    print 'Directory models/{} **DOES NOT EXIST**'.format(MODEL_ID)    

SyntaxError: invalid syntax (<ipython-input-6-a07b125f70e4>, line 4)

In [6]:
## wipe out existing directory
if os.path.exists(MODEL_ID):
    shutil.rmtree(MODEL_ID)
os.mkdir(MODEL_ID)

## Generate dictionary version of model.json

In [7]:
model_dict = {'model_id': MODEL_ID}

### Source data for model

#### Hive Tables
* features
* tables

In [8]:
spark.sql('show tables in features').toPandas()

,database,tableName,isTemporary
0,features,190320_ou_test,False
1,features,190320_test,False
2,features,combined_0127,False
3,features,combined_0320,False
4,features,home_field,False
5,features,line,False
6,features,matchup,False
7,features,rankings,False
8,features,team_history,False
9,features,time_date,False


In [9]:
model_dict['features_tbl'] = 'features.combined_0320'
model_dict['labels_tbl'] = 'labels.combined_0320'

#### Columns from Hive tables
* index: unique identifier in features/labels table (must be in both)
* label column, and indicator of what is a positive label
  * currently not supported: multi-class
  * code will binarize
* list of features

In [10]:
model_dict['index'] = ['game_id']
model_dict['label_col'] = 'did_win'
model_dict['pos_labels'] = [1]
model_dict['neg_labels'] = [-1]
model_dict['features_list'] = ['rankings___h__estim_winrate',
 'rankings___v__offensedvoa',
 'rankings___h__dave_or_wtddvoa',
 'travel___v_travel_from_last_game_decay',
 'rankings___h__offensedvoa',
 'rankings___v__dave_or_wtddvoa',
 'rankings___h__s_t_dvoa',
 'rankings___v__s_t_dvoa',
 'travel___h_travel_from_last_game_decay',
 'rankings___v__defensedvoa',
 'rankings___v__estim_winrate',
 'team_history___v_ovr_wr_past_3_seasons',
 'rankings___h__defensedvoa',
 'team_history___h_ovr_wr_past_3_seasons',
 'weather___wind_chill',
 'home_field___v_visitor__ovr_wr_ytd',
 'home_field___v_visitor__ovr_wr_last_8_games',
 'weather___wind_mph',
 'weather___humidity_pct',
 'team_history___h_ovr_wr_ytd']

model_dict['features_list'].sort()

#### Checks

In [11]:
## assert these fields are of the correct type
assert type(model_dict['index']) is list
assert type(model_dict['label_col']) is str
assert type(model_dict['features_tbl']) is str
assert type(model_dict['features_tbl']) is str
assert type(model_dict['features_list']) is list
assert type(model_dict['pos_labels']) is list

## assert format is schema.table and that
## table exists in hive
for tbl_str in ['features_tbl','features_tbl']:
    schema_and_tbl = model_dict[tbl_str].split('.')
    assert len(schema_and_tbl) == 2
    schema, tbl = schema_and_tbl
    assert spark.sql(
            'show tables in {}'.format(schema)
        ).filter(
            col('tableName') == tbl
        ).count() == 1

feat_cols_set = set(spark.table(model_dict['features_tbl']).columns)
label_cols_set = set(spark.table(model_dict['labels_tbl']).columns)
idx_set = set(model_dict['index'])
feat_set = set(model_dict['features_list'])
label_set = set([model_dict['label_col']])

## assert the chosen columns exist in the
## chosen tables
assert not idx_set - feat_cols_set
assert not idx_set - label_cols_set
assert not feat_set - feat_cols_set
assert not label_set - label_cols_set

## check that positive and negative label values 
## are valid
for label_val in ['pos_labels','neg_labels']:
    assert spark.table(
            model_dict['labels_tbl']
        ).filter(
            col(model_dict['label_col']).isin(model_dict[label_val])
        ).count() > 0
    
## assert that labels and features share identical index
a = spark.table(model_dict['features_tbl']).count()
b = spark.table(model_dict['labels_tbl']).count()
c = spark.table(model_dict['features_tbl']).join(
    spark.table(model_dict['labels_tbl']),
    on=model_dict['index']
).count()
assert a == b
assert a == c

### Cross-Validation Sets
* random seeds for reproducibility
* number of folds for cross-validation (value of <= 1 doesn't do k-fold
* global_dataset_proportions
 * proportion of the data for each of training, scoring only, holdout, and throwaway
 * generated using stratified sampling
* dimensional_dataset_proportions
 * post-processing after global_dataset_proportions
 * idea is to move specific field values, e.g. move certain seasons to the holdout set

#### Optional: use cross-validation data from another model

In [12]:
model_dict['model_cv_to_use'] = None

#### CV parameters, when not using another model CV

#### Sample usage for Dimensional Dataset Proportions

```python
model_dict['dimensional_dataset_proportions'] = {
        'throw_away': [
            {
                'vals': [
                    0
                ], 
                'dim': 'is_home',
                'prop_to_move': 1.0, 
                'from_groups': [
                    'in_training',
                    'holdout',
                    'scoring_only'
                ]
            }
        ]
    }
```

In [13]:
model_dict['kfold_seed'] = 99
model_dict['dataset_seed'] = 9
model_dict['kfolds'] = 5
model_dict['strata_cols'] = ['did_cover_sbr','week_id']
model_dict['holdout_set'] = {
    'store_to_disk': False,
    'score_using_full_model': False 
}

model_dict['global_dataset_proportions'] = {
        'in_training': 1.,
        'holdout': 0,
        'throw_away': 0,
        'scoring_only': 0
    }

# DEFAULT: model_dict['dimensional_dataset_proportions'] = {}
model_dict['dimensional_dataset_proportions'] = {
        'throw_away': [
            {
                'vals': [
                    1, 2, 3, 4, 17, 18, 19, 20, 21, 22
                ], 
                'dim': 'week_id',
                'prop_to_move': 1.0, 
                'from_groups': [
                    'in_training',
                    'holdout',
                    'scoring_only'
                ]
            }
        ]
    }



#### Checks

In [14]:
if model_dict['model_cv_to_use']:
    assert type(model_dict['model_cv_to_use']) in [str, unicode]
    assert os.path.exists(model_dict['model_cv_to_use'])
else:
    ## assert the data structures/types are correct
    assert type(model_dict['kfold_seed']) is int
    assert type(model_dict['dataset_seed']) is int
    assert type(model_dict['kfolds']) is int
    assert type(model_dict['strata_cols']) is list
    assert type(model_dict['global_dataset_proportions']) is dict
    assert type(model_dict['dimensional_dataset_proportions']) is dict
    assert type(model_dict['holdout_set']) is dict

    ## assert strata cols are present in the labels table
    assert not set(model_dict['strata_cols']) - label_cols_set

    dataset_types = set(['in_training','holdout','throw_away','scoring_only'])
    global_datasets = model_dict['global_dataset_proportions']
    dim_datasets = model_dict['dimensional_dataset_proportions']

    ## assert global_dataset_proportions has all possible dataset types
    assert set(global_datasets.keys()) == dataset_types
    ## values are proportions that must sum to 1
    assert sum(global_datasets.values()) == 1
    ## assert that the keys are valid dataset types
    assert not set(dim_datasets.keys()) - dataset_types
    ## assert the following (in order of assertion block):
    ## (1) each value is a list
    ## (2) each element of the list is a dict
    ## (3) each dict has the 5 required keys
    ## (4) the "dim" field is in the strata columns 
    ## (5) "prop_to_move" field is [0, 1]
    ## (6) "from_groups" are in the possible dataset types
    for k, dim_list in dim_datasets.iteritems():
        assert (type(dim_list)) is list
        for entry in dim_list:
            assert type(entry) is dict
            assert set(entry.keys()) \
                    == set(['vals','dim','prop_to_move','from_groups'])
            assert entry['dim'] in model_dict['strata_cols']
            assert 0 <= entry['prop_to_move'] <= 1
            assert not set(entry['from_groups']) - dataset_types

    ## assert holdout set has 2 keys (store_to_disk, score_using_full_model)
    ## and the corresponding values are boolean
    assert set(model_dict['holdout_set'].keys()) \
            == set(['store_to_disk','score_using_full_model'])
    assert len(filter(
        lambda x: type(x) is not bool, 
        model_dict['holdout_set'].values()
    )) == 0
    ## if holdout data isn't stored, it can't be scored
    assert not (model_dict['holdout_set']['store_to_disk'] is False) \
                & (model_dict['holdout_set']['score_using_full_model'] is True)

### Model Choice
* package/class name as a string
* parameters as a dictionary

#### sklearn basic

In [15]:
model_dict['model'] = 'sklearn.ensemble.GradientBoostingClassifier'
model_dict['model_params'] = {
    'learning_rate': 0.1, 
    'n_estimators': 200, 
    'max_features': 'auto', 
    'subsample': 0.9, 
    'random_state': 9, 
    'max_depth': 12, 
}

#### xgboost basic

In [16]:
model_dict['model'] = 'xgboost.XGBClassifier'
model_dict['model_params'] = {
        'n_jobs': 1, 
        'learning_rate': 0.1, 
        'n_estimators': 100, 
        'max_features': 'auto', 
        'booster': 'gbtree', 
        'silent': True, 
        'nthread': None, 
        'subsample': 0.5, 
        'random_state': 9, 
        'objective': 'binary:logistic', 
        'max_depth': 6, 
        'gamma': 0
    }

#### Checks

In [17]:
## test that model object can be created
## from model inputs
try:
    import importlib

    model_class_str = model_dict['model']
    model_obj_path = '.'.join(model_class_str.split('.')[:-1])
    model_name = model_class_str.split('.')[-1]
    model_package = importlib.import_module(model_obj_path)
    model_class = getattr(model_package, model_name)
    _ = model_class(**model_dict['model_params'])
except Exception as e:
    e

### Write out model.json file

In [18]:
model_json_path = '{}/model.json'.format(model_dict['model_id'])
assert os.path.exists(model_dict['model_id'])
assert not os.path.exists(model_json_path)

with open(model_json_path,'w') as w:
    json.dump(model_dict, w, indent=4)

## Generate dictionary version of plots.json

In [19]:
plots_dict = {'model_id': MODEL_ID}

### Plot Labels
* labels --> names (note: keys should be strings)
* name for success rate

In [20]:
plots_dict['label_map'] = {
    '1': 'Won',
    '0': 'Lost'
}
plots_dict['success_name'] = 'Win Rate'

#### Checks

In [21]:
assert type(plots_dict['label_map']) is dict
assert type(plots_dict['success_name']) is str
assert set(plots_dict['label_map'].keys()) == set(['0','1'])

### Bins to plot
* plot_bins: 
   * Number of bins to plot (i.e. number of bars on the bar chart)
* bin_types:
   * "Bin" puts scores into uniform bins, e.g. [0, 0.10], (0.10, 0.20], ..., (0.9, 1.0]
   * "Percentile" bins scores into ntiles determined by plot_bins

In [22]:
plots_dict['bin_types'] = ['Bin', 'Percentile']
plots_dict['plot_bins'] = [10, 100]

#### Checks

In [23]:
## currently only supports "Bin" and "Percentile"
assert not set(plots_dict['bin_types']) - set(['Bin','Percentile'])
## all plot bins values should be ints
assert plots_dict['plot_bins'] == map(int, plots_dict['plot_bins'])
## ensure all bins values are in [2, 1000]
assert filter(
        lambda x: 2 <= x <= 1000, plots_dict['plot_bins']
    )   == plots_dict['plot_bins']

### Threshold Metrics to Plot
* metrics evaluated at each of 100 score threshold points
* currently only supports Accuracy and F1

In [24]:
plots_dict['threshold_metrics'] = ['Accuracy','F1']

#### Checks

In [25]:
assert type(plots_dict['threshold_metrics']) is list
## currently only supports Accuracy and F1
assert not set(plots_dict['threshold_metrics']) - set(['Accuracy','F1'])

### Write out plots.json

In [26]:
plots_json_path = '{}/plots.json'.format(model_dict['model_id'])
assert not os.path.exists(plots_json_path)

with open(plots_json_path,'w') as w:
    json.dump(plots_dict, w, indent=4)

## Execute pipeline

In [27]:
MODEL_ID
!source ~/.bashrc && \
    unset PYSPARK_PYTHON && \
    unset PYSPARK_DRIVER_PYTHON && \
    unset PYSPARK_DRIVER_PYTHON_OPTS && \
    cd /Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/ && \
    model_pipeline/model_pipeline.sh {MODEL_ID}

'0320_with_rankings_winner_20feats_noml'

Check JSON files

real	0m15.191s
user	0m35.433s
sys	0m2.689s

Cross-validation data

real	0m25.279s
user	1m33.174s
sys	0m9.026s

Train and score

real	0m2.283s
user	0m1.351s
sys	0m0.918s

Evaluate and plot

real	0m1.358s
user	0m0.942s
sys	0m0.394s


In [28]:
!cat /Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/models/{MODEL_ID}/logs/out

JSON configuration files passed checks.
cv sets wrote successfully.
Traceback (most recent call last):
  File "model_pipeline/eval_plot.py", line 524, in <module>
    importance = get_feat_importance_df()
  File "model_pipeline/eval_plot.py", line 408, in get_feat_importance_df
    assert imp_files
AssertionError


In [29]:
!open {MODEL_ID}

## Hyperparameter Optimization

In [3]:
ROOT='/Users/joshplotkin/Dropbox/data_science/'\
    'modeling-football-outcomes/models'
os.chdir(ROOT)

In [4]:
MODEL_ID='0320_with_rankings_winner_20feats_noml'

In [5]:
from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC

In [6]:
from sklearn.metrics import roc_auc_score

In [7]:
sys.path.append('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/model_pipeline')

In [8]:
from train_score_functions import get_model_obj, store_feature_importance, check_bad_values, cv_train, cv_score, score_holdout_set, store_models

In [9]:
## 
model_dict = json.load(open('{}/model.json'.format(MODEL_ID)))
os.chdir(MODEL_ID)

In [10]:
training = pd.read_csv('cv_data/training.csv')
scoring_only = pd.read_csv('cv_data/scoring_only.csv')

# if not os.path.exists('serialized_models'): 
#     os.mkdir('serialized_models')
# if not os.path.exists('scores'): 
#     os.mkdir('scores')
# if not os.path.exists('stats'): 
#     os.mkdir('stats')
#     os.mkdir('stats/reported')

In [11]:
import importlib
def xgb_crossval(n_estimators, max_depth):
    model_dict['model_params'] = {
                                 u'booster': u'gbtree',
                                 u'gamma': 0,
                                 u'learning_rate': 0.1,
                                 u'max_depth': int(max_depth),
                                 u'max_features': u'auto',
                                 u'n_estimators': int(n_estimators),
                                 u'n_jobs': 1,
                                 u'nthread': None,
                                 u'objective': u'binary:logistic',
                                 u'random_state': 9,
                                 u'silent': True,
                                 u'subsample': 0.5}

    model_class_str = model_dict['model']
    model_obj_path = '.'.join(model_class_str.split('.')[:-1])
    model_name = model_class_str.split('.')[-1]
    model_package = importlib.import_module(model_obj_path)
    model_obj = getattr(model_package, model_name)
    library = model_class_str.split('.')[0]


    training = pd.read_csv('cv_data/training.csv')
    scoring_only = pd.read_csv('cv_data/scoring_only.csv')
#     model_obj, library = get_model_obj(model_dict)
    library = 'xgboost'
    ## in memory is faster when possible
    ## spark can train/score on larger data when needed
    if library in ['sklearn','xgboost']:
        training_scoring_dict = cv_train(model_dict, training, 
                                         scoring_only, model_obj)

        store_models(library, training_scoring_dict)

        scores_df = cv_score(model_dict, training_scoring_dict)
        scores_df[['label','score']].to_csv('scores/reported_scores.csv')

        if model_dict['holdout_set']['score_using_full_model']:
            holdout = pd.read_csv('cv_data/holdout.csv')
            holdout = score_holdout_set(model_dict, training_scoring_dict, 
                                        holdout)
            holdout[['label','score']].to_csv('scores/holdout_scores.csv')

    score_pred = scores_df[['label','score']]\
                    .reset_index(drop=False)\
                    .set_index('game_id')
    
    print(roc_auc_score(score_pred['label'], score_pred['score'])    )
    return -roc_auc_score(score_pred['label'], score_pred['score'])    

In [ ]:
optimizer = BayesianOptimization(
    f=xgb_crossval,
    pbounds={
        "n_estimators": (100, 250),
        "max_depth": (3, 6)
    }
)

optimizer.maximize(n_iter=10)

print("Final result:", optimizer.max)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
0.6589704994529151
|  1        | -0.659    |  5.601    |  214.0    |
0.6692781164864653
|  2        | -0.6693   |  5.697    |  104.4    |
0.6708230329173464
|  3        | -0.6708   |  5.539    |  101.2    |
0.6687374541225214
|  4        | -0.6687   |  3.917    |  232.4    |


In [2]:
os.chdir('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/models/')



['0128_with_features',
 '0115_test_ou',
 '0115_test_ou_sklearn',
 '0228_with_rankings_features_winner',
 '0228_with_rankings_winner_24feats_noml',
 '.DS_Store',
 '0114_test',
 '0128_with_features_sklearn',
 '0115_test_spread_sklearn',
 '0114_test_ou',
 '0228_with_rankings_features_spread',
 '0112_test',
 '0228_with_rankings_winner_28feats_noml',
 '0228_with_rankings_winner_20feats_noml',
 '0127_with_features',
 '0320_with_rankings_winner_20feats_noml',
 '0127_with_features_sklearn',
 '0228_with_rankings_winner_33feats_noml',
 '0128_with_features_sklearn_spread',
 '0228_with_rankings_winner_44feats_noml']

In [10]:
import numpy as np

In [23]:
dirs = os.listdir('.')

def is_int(d):
    try:
        int(d)
        return True
    except:
        return False

min_id = min(set(np.arange(100000)) - set(map(int, (filter(is_int, dirs)))))
min_id_str = '{:06d}'.format(min_id)

In [24]:
min_id_str

'000000'

In [28]:
config = json.load(open('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/06_model_development/config.json'))

In [33]:
variable_params = {k:v for k,v in config.items() if type(v) is list}
fixed_params = {k:v for k,v in config.items() if type(v) is not list}

In [35]:
import importlib

model_class_str = 'xgboost.XGBClassifier'
model_obj_path = '.'.join(model_class_str.split('.')[:-1])
model_name = model_class_str.split('.')[-1]
model_package = importlib.import_module(model_obj_path)
model_class = getattr(model_package, model_name)
library = model_class_str.split('.')[0]


In [38]:
model_obj_path

'xgboost'

In [39]:
model_name

'XGBClassifier'

In [40]:
model_package

<module 'xgboost' from '/Users/joshplotkin/anaconda3/lib/python3.7/site-packages/xgboost/__init__.py'>

In [41]:
model_class

xgboost.sklearn.XGBClassifier

In [43]:
import xgboost

xgboost.sklearn.XGBClassifier

xgboost.sklearn.XGBClassifier

In [52]:
fields = '''n_estimators, max_depth, eta, gamma_min_split_loss_exp, min_child_weight_exp,
    max_delta_step, subsample, colsample_bytree, colsample_bylevel, 
    colsample_bynode, lambda_exp, alpha_exp'''.replace('\n','').strip()

print(', '.join(sorted(list(map(lambda x: x.strip(), fields.split(', '))))))

alpha_exp, colsample_bylevel, colsample_bynode, colsample_bytree, eta, gamma_min_split_loss_exp, lambda_exp, max_delta_step, max_depth, min_child_weight_exp, n_estimators, subsample


In [62]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [69]:
optimizer.max['target']

{'target': 0.69695173858566,
 'params': {'alpha_exp': 5.0,
  'colsample_bylevel': 1.0,
  'colsample_bynode': 0.1,
  'colsample_bytree': 1.0,
  'eta': 0.0,
  'gamma_min_split_loss_exp': -5.0,
  'lambda_exp': -5.0,
  'max_delta_step': 1.0,
  'max_depth': 1.0,
  'min_child_weight_exp': -5.0,
  'n_estimators': 212.63585101657023,
  'subsample': 1.0}}

In [70]:
optimizer.res

[{'target': 0.6673151501067895,
  'params': {'alpha_exp': -3.8885835051818507,
   'colsample_bylevel': 0.9858268686075451,
   'colsample_bynode': 0.6796187459811552,
   'colsample_bytree': 0.8868980816343365,
   'eta': 0.07044895111335936,
   'gamma_min_split_loss_exp': -0.2822834266077674,
   'lambda_exp': -2.3699258498323026,
   'max_delta_step': 76.52953445742872,
   'max_depth': 13.057720340930597,
   'min_child_weight_exp': -3.822320390164472,
   'n_estimators': 725.6134924560853,
   'subsample': 0.5559879911555794}},
 {'target': 0.47486795810625704,
  'params': {'alpha_exp': 3.2537842188727915,
   'colsample_bylevel': 0.11083861868329276,
   'colsample_bynode': 0.8660938048524637,
   'colsample_bytree': 0.32602897155355265,
   'eta': 0.20405242804465673,
   'gamma_min_split_loss_exp': 4.56052802422985,
   'lambda_exp': -4.769596075855878,
   'max_delta_step': 19.498509985913472,
   'max_depth': 24.695969107717147,
   'min_child_weight_exp': -1.9018697653623304,
   'n_estimators':

In [66]:
optimizer._events
optimizer._gp
optimizer._prime_queue
optimizer._prime_subscriptions
optimizer._queue
optimizer._random_state
optimizer._space
optimizer._verbose
optimizer.dispatch
optimizer.get_subscribers
optimizer.max
optimizer.maximize
optimizer.probe
optimizer.register
optimizer.res
optimizer.set_bounds
optimizer.set_gp_params
optimizer.space
optimizer.subscribe
optimizer.suggest
optimizer.unsubscribe

{'optmization:start': {<bayes_opt.observer.ScreenLogger at 0x1a239ac320>: <bound method ScreenLogger.update of <bayes_opt.observer.ScreenLogger object at 0x1a239ac320>>},
 'optmization:step': {<bayes_opt.observer.ScreenLogger at 0x1a239ac320>: <bound method ScreenLogger.update of <bayes_opt.observer.ScreenLogger object at 0x1a239ac320>>},
 'optmization:end': {<bayes_opt.observer.ScreenLogger at 0x1a239ac320>: <bound method ScreenLogger.update of <bayes_opt.observer.ScreenLogger object at 0x1a239ac320>>}}

GaussianProcessRegressor(alpha=0.001, copy_X_train=True,
             kernel=Matern(length_scale=1, nu=2.5), n_restarts_optimizer=1,
             normalize_y=True, optimizer='fmin_l_bfgs_b',
             random_state=<mtrand.RandomState object at 0x1a239fc360>)

<bound method BayesianOptimization._prime_queue of <bayes_opt.bayesian_optimization.BayesianOptimization object at 0x1a239ca780>>

<bound method BayesianOptimization._prime_subscriptions of <bayes_opt.bayesian_optimization.BayesianOptimization object at 0x1a239ca780>>

99

...

<bound method BayesianOptimization.set_bounds of <bayes_opt.bayesian_optimization.BayesianOptimization object at 0x1a239ca780>>

<bound method BayesianOptimization.set_gp_params of <bayes_opt.bayesian_optimization.BayesianOptimization object at 0x1a239ca780>>

<bound method Observable.subscribe of <bayes_opt.bayesian_optimization.BayesianOptimization object at 0x1a239ca780>>

<bound method BayesianOptimization.suggest of <bayes_opt.bayesian_optimization.BayesianOptimization object at 0x1a239ca780>>

<bound method Observable.unsubscribe of <bayes_opt.bayesian_optimization.BayesianOptimization object at 0x1a239ca780>>

In [57]:
dir(optimizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_events',
 '_gp',
 '_prime_queue',
 '_prime_subscriptions',
 '_queue',
 '_random_state',
 '_space',
 '_verbose',
 'dispatch',
 'get_subscribers',
 'max',
 'maximize',
 'probe',
 'register',
 'res',
 'set_bounds',
 'set_gp_params',
 'space',
 'subscribe',
 'suggest',
 'unsubscribe']

### SKOPT

In [144]:
from collections import OrderedDict
import json
import pandas as pd
from sklearn.metrics import roc_auc_score
from skopt import Optimizer
from skopt.learning import ExtraTreesRegressor
import sys

sys.path.append('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/model_pipeline')
from train_score_functions import get_model_obj, store_feature_importance, check_bad_values, cv_train, cv_score, score_holdout_set, store_models

In [143]:
def xgb_crossval(variable_params):
    training = source_model['training']
    scoring_only = source_model['scoring_only']
    model_dict = source_model['config']

    model_dict['model_params'] = {
        ## strings
        'booster': config['booster'],
        'tree_method': config['tree_method'],

        ## base
        'n_estimators': int(variable_params['n_estimators']),
        'eta': variable_params['eta'],

        ## sampling
        'colsample_bytree': variable_params['colsample_bytree'],
        'colsample_bylevel': variable_params['colsample_bylevel'],
        'colsample_bynode': variable_params['colsample_bynode'],
        'subsample': variable_params['subsample'],

        ## tree splitting
        'max_depth': int(variable_params['max_depth']),
        'gamma': 10 ** variable_params['gamma_min_split_loss_exp'],
        'min_child_weight': 10 ** variable_params['min_child_weight_exp'],
        'max_delta_step': variable_params['max_delta_step'],

        ## regularization
        'lambda': 10 ** variable_params['lambda_exp'],
        'alpha': 10 ** variable_params['alpha_exp'],
    }

    model_obj = xgboost.sklearn.XGBClassifier
    training_scoring_dict = cv_train(model_dict, training,
                                     scoring_only, model_obj)
    scores_df = cv_score(model_dict, training_scoring_dict)
    score_pred = scores_df[['label', 'score']] \
        .reset_index(drop=False) \
        .set_index('game_id')

    return -roc_auc_score(score_pred['label'], score_pred['score'])

In [133]:
def separate_fixed_variable_params(config):
    variable_params = {k: v for k, v in config.items() if type(v) is list}
    fixed_params = {k: v for k, v in config.items() if type(v) is not list}

    # need ordered variable params so it lines up with function call
    variable_params_ordered = OrderedDict()
    for ksort in sorted(variable_params.keys()):
        variable_params_ordered[ksort] = variable_params[ksort]

    return fixed_params, variable_params_ordered

In [134]:
config = json.load(open('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/06_model_development/config.json'))
source_model = {
    'config': json.load(open('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/models/0320_with_rankings_winner_20feats_noml/model.json')),
    'training': pd.read_csv('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/models/0320_with_rankings_winner_20feats_noml/cv_data/training.csv'),
    'scoring_only': pd.read_csv('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/models/0320_with_rankings_winner_20feats_noml/cv_data/scoring_only.csv'),
}

In [135]:
fixed_params, variable_params = separate_fixed_variable_params(config)

In [136]:
variable_df = pd.DataFrame.from_dict(variable_params, orient='index')
variable_df['range'] = variable_df.apply(lambda x: tuple(x), axis=1)
variable_df = variable_df.sort_index()

In [137]:
# Optimizer(
#     dimensions,
#     base_estimator='gp',
#     n_random_starts=None,
#     n_initial_points=10,
#     acq_func='gp_hedge',
#     acq_optimizer='auto',
#     random_state=None,
#     acq_func_kwargs=None,
#     acq_optimizer_kwargs=None,
# )

opt = Optimizer(
    variable_df['range'].tolist(), 
    base_estimator=config.get('base_estimator', 'ET'), 
    acq_optimizer=config.get(acq_optimizer, 'sampling'),
    n_random_starts=config.get('n_random_starts', None),
    n_initial_points=config.get('n_initial_points', 10),
    acq_func=config.get('acq_func', 'gp_hedge'),
    random_state=config.get('random_state', None),
    acq_func_kwargs=config.get('acq_func_kwargs', None),
    acq_optimizer_kwargs=config.get('acq_optimizer_kwargs', None)
)
next_x = opt.ask()

variable_df['next'] = next_x
f_val = xgb_crossval(variable_df['next'].to_dict())
opt.tell(next_x, f_val)

# f_val = objective(next_x)
# opt.tell(next_x, f_val)


[3.000379046582502, 0.759879100666941, 0.9779854916816486, 0.8471854052241471, 0.5006582129540089, 4.325958947644152, 4.960041311226995, 89.59564427267352, 16.058709554729873, -3.1066356008556335, 904.3028206886041, 0.7934957012902615]


In [147]:
next_x = opt.ask()
variable_df['next'] = next_x
f_val = xgb_crossval(variable_df['next'].to_dict())
print(f_val)
opt.tell(next_x, f_val)

0.6904807224089953


          fun: 0.47486795810625704
    func_vals: array([0.47486796, 0.69048072])
       models: []
 random_state: <mtrand.RandomState object at 0x10f624e58>
        space: Space([Real(low=-5.0, high=5.0, prior='uniform', transform='identity'),
       Real(low=0.1, high=1.0, prior='uniform', transform='identity'),
       Real(low=0.1, high=1.0, prior='uniform', transform='identity'),
       Real(low=0.1, high=1.0, prior='uniform', transform='identity'),
       Real(low=0.0, high=1.0, prior='uniform', transform='identity'),
       Real(low=-5.0, high=5.0, prior='uniform', transform='identity'),
       Real(low=-5.0, high=5.0, prior='uniform', transform='identity'),
       Real(low=1.0, high=100.0, prior='uniform', transform='identity'),
       Real(low=1.0, high=30.0, prior='uniform', transform='identity'),
       Real(low=-5.0, high=5.0, prior='uniform', transform='identity'),
       Real(low=10.0, high=1000.0, prior='uniform', transform='identity'),
       Real(low=0.1, high=1.0, prio

In [148]:
fixed_params.values()

{'model': 'xgboost.XGBClassifier',
 'booster': 'gbtree',
 'tree_method': 'auto',
 'init_points': 1,
 'n_iter': 100000,
 'alpha': 0.001,
 'n_restarts_optimizer': 1}

In [152]:
header = list(variable_df.index.tolist() + list(fixed_params.keys()) + ['auc'])
header

['alpha_exp',
 'colsample_bylevel',
 'colsample_bynode',
 'colsample_bytree',
 'eta',
 'gamma_min_split_loss_exp',
 'lambda_exp',
 'max_delta_step',
 'max_depth',
 'min_child_weight_exp',
 'n_estimators',
 'subsample',
 'model',
 'booster',
 'tree_method',
 'init_points',
 'n_iter',
 'alpha',
 'n_restarts_optimizer',
 'auc']

In [155]:
run_auc = f_val

In [158]:
dict(zip(header, variable_df['next'].tolist() + list(fixed_params.values()) + [run_auc]))

{'alpha_exp': 4.9070593390429575,
 'colsample_bylevel': 0.519504121102633,
 'colsample_bynode': 0.9828574888538592,
 'colsample_bytree': 0.5232879778604249,
 'eta': 0.29300847122314294,
 'gamma_min_split_loss_exp': 1.5256659146658995,
 'lambda_exp': -4.636497806275496,
 'max_delta_step': 31.869710344785254,
 'max_depth': 21.663360380659693,
 'min_child_weight_exp': 1.3817883221833291,
 'n_estimators': 646.825737734592,
 'subsample': 0.7502194283577838,
 'model': 'xgboost.XGBClassifier',
 'booster': 'gbtree',
 'tree_method': 'auto',
 'init_points': 1,
 'n_iter': 100000,
 'alpha': 0.001,
 'n_restarts_optimizer': 1,
 'auc': 0.6904807224089953}

In [160]:
mdl = '/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/models/0228_with_rankings_winner_44feats_noml'

config = json.load(open('/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/06_model_development/config.json'))
source_model = {
    'config': json.load(open(f'{mdl}/model.json')),
    'training': pd.read_csv(f'{mdl}/cv_data/training.csv'),
    'scoring_only': pd.read_csv(f'{mdl}/cv_data/scoring_only.csv'),
}

In [163]:
source_model['config']['features_list']

['home_field___h_consecutive_home',
 'home_field___h_home__ovr_wr_last_4_games',
 'home_field___h_home__ovr_wr_last_8_games',
 'home_field___h_home__ovr_wr_ytd',
 'home_field___v_consecutive_visitor',
 'home_field___v_visitor__ovr_wr_last_4_games',
 'home_field___v_visitor__ovr_wr_last_8_games',
 'home_field___v_visitor__ovr_wr_ytd',
 'matchup___h_wr_past_3_seasons',
 'matchup___is_same_division',
 'rankings___h__dave_or_wtddvoa',
 'rankings___h__defensedvoa',
 'rankings___h__estim_winrate',
 'rankings___h__offensedvoa',
 'rankings___h__s_t_dvoa',
 'rankings___v__dave_or_wtddvoa',
 'rankings___v__defensedvoa',
 'rankings___v__estim_winrate',
 'rankings___v__offensedvoa',
 'rankings___v__s_t_dvoa',
 'team_history___h_ovr_wr_past_2_seasons',
 'team_history___h_ovr_wr_past_3_seasons',
 'team_history___h_ovr_wr_ytd',
 'team_history___v_ovr_wr_past_2_seasons',
 'team_history___v_ovr_wr_past_3_seasons',
 'team_history___v_ovr_wr_ytd',
 'time_date___game_time_eastern',
 'time_date___is_sunday